# 06. Agent Evaluation (Agent 평가)

이 노트북에서는 배포된 Agent의 품질과 성능을 평가합니다.

## 목표

- Azure AI Evaluation SDK를 사용한 Agent 평가
- 성능, 정확도, 안전성 메트릭 수집
- 평가 결과 분석 및 시각화

## 사전 요구사항

1. Notebook 01-03 완료 (Azure 리소스 및 Agent 배포)
2. `config.json` 파일 존재
3. Azure AI Project 접근 권한

---

---

## ⚙️ 시작하기 전에 (Before You Start)

**Python 커널을 선택하세요:**

1. 노트북 오른쪽 상단의 **"커널 선택 (Select Kernel)"** 클릭
2. **"Python Environments..."** 선택
3. **`.venv (Python 3.x.x)`** 선택 (프로젝트 루트에 생성된 가상환경)

> 💡 **GitHub Codespaces**: Codespaces에서는 자동으로 `.venv` 환경이 생성됩니다.  
> 만약 `.venv`가 보이지 않으면 터미널에서 `python -m venv .venv`로 생성하세요.

---

## 1. 환경 설정 및 Config 로드

이전 노트북에서 생성한 설정을 로드합니다.

In [ ]:
import json
import sys
import subprocess
from pathlib import Path

# config.json 로드
config_path = Path("config.json")
if not config_path.exists():
    raise FileNotFoundError("config.json not found. Please run notebooks 01-03 first.")

with open(config_path, 'r') as f:
    config = json.load(f)

# 주요 변수 추출
PROJECT_CONNECTION_STRING = config.get("project_connection_string", "")
simple_project_conn = PROJECT_CONNECTION_STRING.split(';')[0] if PROJECT_CONNECTION_STRING else ""

print("=== Configuration Loaded ===")
print(f"Resource Group: {config.get('resource_group')}")
print(f"Location: {config.get('location')}")
print(f"Project: {simple_project_conn}")
print(f"Model: {config.get('model_deployment_name')}")
print("\n" + "="*50)

## 2. Azure AI Evaluation 패키지 설치

Agent 평가에 필요한 패키지를 설치합니다.

In [ ]:
# Azure AI Evaluation 패키지 설치
print("=== Installing Azure AI Evaluation Package ===\n")

result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "azure-ai-evaluation"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ azure-ai-evaluation installed successfully")
else:
    print(f"⚠️  Installation warning: {result.stderr}")
    print("   Proceeding anyway...")

print("\n" + "="*50)

## 3. Evaluation 개요

### Evaluation 유형

**Performance Evaluators (성능 평가):**
- **Intent Resolution**: Agent가 사용자 의도를 올바르게 파악했는지 평가
- **Tool Call Accuracy**: Agent가 올바른 도구를 정확하게 호출했는지 평가  
- **Task Adherence**: Agent가 지시사항을 충실히 따랐는지 평가

**Safety Evaluators (안전성 평가):**
- **Content Safety**: 부적절한 콘텐츠(폭력, 혐오 등) 포함 여부 평가
- **Indirect Attack**: 간접적인 악의적 공격 시도 감지
- **Code Vulnerability**: 생성된 코드의 보안 취약점 평가

### Evaluation 프로세스

1. 테스트 쿼리 생성
2. Agent 생성 및 쿼리 실행
3. 응답 수집 및 메트릭 측정
4. Evaluator로 품질 평가
5. 결과 저장 및 분석
6. Agent 정리

---

## 4. 테스트 쿼리 생성

Agent의 다양한 기능을 테스트할 쿼리를 생성합니다.

In [ ]:
# evals 디렉토리 생성
import os
from pathlib import Path

evals_dir = Path("evals")
evals_dir.mkdir(exist_ok=True)

print("=== Creating Evaluation Test Queries ===\n")

# 테스트 쿼리 정의
eval_queries = [
    {
        "query": "안녕하세요. 가족 여행지를 추천받고 싶어요.",
        "ground-truth": "Agent는 인사에 응답하고, 여행지 추천을 위해 Research Agent를 호출해야 합니다."
    },
    {
        "query": "부산의 현재 날씨를 알려주세요. 온도와 체감온도를 포함해주세요.",
        "ground-truth": "부산의 현재 날씨 정보를 정확하게 제공해야 하며, 온도와 체감온도를 모두 포함해야 합니다."
    },
    {
        "query": "제주도에서 가족과 함께 즐길 수 있는 여행지는 어디인가요?",
        "ground-truth": "제주도의 가족 친화적인 여행지를 검색하여 추천해야 합니다. 자연 명소, 체험 활동, 접근성 등을 고려한 정보를 제공합니다."
    },
    {
        "query": "서핑할 수 있는 해변을 추천해주세요.",
        "ground-truth": "서핑이 가능한 한국의 해변 여행지를 검색하여 추천해야 합니다. 양양, 부산 등의 서핑 명소 정보를 제공합니다."
    },
    {
        "query": "가을에 가기 좋은 단풍 명소는 어디인가요?",
        "ground-truth": "가을 계절에 방문하기 좋은 단풍 명소를 검색하여 추천해야 합니다. 내장산, 설악산 등의 자연 명소 정보를 제공합니다."
    }
]

# JSON 파일로 저장
eval_queries_path = evals_dir / "eval-queries.json"
with open(eval_queries_path, 'w', encoding='utf-8') as f:
    json.dump(eval_queries, f, indent=2, ensure_ascii=False)

print(f"✅ Created {eval_queries_path}")
print(f"\n📋 Test Queries ({len(eval_queries)} total):\n")

for i, query in enumerate(eval_queries, 1):
    print(f"   {i}. {query['query'][:60]}...")

print("\n💡 각 쿼리는 Agent의 다른 기능을 테스트합니다:")
print("   • 일반 대화 및 여행 의도 파악")
print("   • 날씨 조회 (Tool 기능)")
print("   • 여행지 지식 검색 (RAG 기능)")

print("\n" + "="*60)

## 5. Agent Evaluation 실행

평가용 Agent를 생성하고 테스트 쿼리를 실행한 후, Evaluator로 품질을 평가합니다.

### 참고사항

- **실행 시간**: 약 2-3분 소요
- **리전 제약**: eastus 리전에서는 일부 Safety Evaluator가 지원되지 않습니다
- **권한 이슈**: Azure AI Project의 스토리지 권한이 필요할 수 있습니다

In [ ]:
# Agent Evaluation 실행
import time
from pathlib import Path
from urllib.parse import urlparse
from azure.ai.agents.models import RunStatus, MessageRole
from azure.ai.projects import AIProjectClient
from azure.ai.evaluation import (
    AIAgentConverter, evaluate, ToolCallAccuracyEvaluator, IntentResolutionEvaluator, 
    TaskAdherenceEvaluator
)
from azure.identity import DefaultAzureCredential

print("=== Running Agent Evaluation ===\n")

# 파일 경로 설정
current_dir = Path(".")
evals_dir = current_dir / "evals"
eval_queries_path = evals_dir / "eval-queries.json"
eval_input_path = evals_dir / "eval-input.jsonl"
eval_output_path = evals_dir / "eval-output.json"

# 환경 변수 로드 (이미 config에서 로드됨)
project_endpoint = simple_project_conn
parsed_endpoint = urlparse(project_endpoint)
model_endpoint = f"{parsed_endpoint.scheme}://{parsed_endpoint.netloc}"
deployment_name = config.get("model_deployment_name", "gpt-4o")

print(f"📋 Configuration:")
print(f"   Project: {project_endpoint}")
print(f"   Model: {deployment_name}")
print(f"   Test Queries: {eval_queries_path}")
print(f"\n")

# Initialize AIProjectClient
print("🔌 Connecting to AI Project...")
credential = DefaultAzureCredential()
ai_project = AIProjectClient(
    credential=credential,
    endpoint=project_endpoint,
    api_version="2025-05-15-preview"  # Evaluations require preview API
)
print("✅ Connected\n")

# Evaluation용 Agent 생성
print("🤖 Creating Evaluation Agent...")
eval_agent = ai_project.agents.create_agent(
    model=deployment_name,
    name="Evaluation Agent",
    instructions="""You are a helpful travel and weather assistant.
    
You can help users with:
1. Travel recommendations and destination information
2. Weather information for any city
3. General travel planning advice

Be friendly, informative, and provide detailed responses."""
)
print(f"✅ Agent created: {eval_agent.name} (ID: {eval_agent.id})\n")

# Setup evaluation config
api_version = config.get("api_version", "2024-08-01-preview")
model_config = {
    "azure_deployment": deployment_name,
    "azure_endpoint": model_endpoint,
    "api_version": api_version,
}

thread_data_converter = AIAgentConverter(ai_project)

# 테스트 쿼리 실행 및 evaluation input 준비
print("="*70)
print("📝 Executing Test Queries\n")

with open(eval_queries_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

print(f"   Total queries: {len(test_data)}\n")

with open(eval_input_path, "w", encoding="utf-8") as f:
    for idx, row in enumerate(test_data, 1):
        query_text = row.get("query")
        print(f"   [{idx}/{len(test_data)}] {query_text[:50]}...")
        
        # 새 스레드 생성 (각 쿼리를 격리)
        thread = ai_project.agents.threads.create()
        
        # 사용자 쿼리 생성
        ai_project.agents.messages.create(
            thread.id, role=MessageRole.USER, content=query_text
        )
        
        # Agent 실행 및 성능 측정
        start_time = time.time()
        run = ai_project.agents.runs.create_and_process(
            thread_id=thread.id, agent_id=eval_agent.id
        )
        end_time = time.time()
        
        if run.status != RunStatus.COMPLETED:
            print(f"      ⚠️  Run failed: {run.last_error or 'Unknown error'}")
            continue
        
        # 운영 메트릭 수집
        operational_metrics = {
            "server-run-duration-in-seconds": (
                run.completed_at - run.created_at
            ).total_seconds(),
            "client-run-duration-in-seconds": end_time - start_time,
            "completion-tokens": run.usage.completion_tokens,
            "prompt-tokens": run.usage.prompt_tokens,
            "ground-truth": row.get("ground-truth", '')
        }
        
        # Thread 데이터 + 운영 메트릭을 evaluation input에 추가
        evaluation_data = thread_data_converter.prepare_evaluation_data(thread_ids=thread.id)
        eval_item = evaluation_data[0]
        eval_item["metrics"] = operational_metrics
        f.write(json.dumps(eval_item, ensure_ascii=False) + "\n")
        
        print(f"      ✅ Completed in {operational_metrics['client-run-duration-in-seconds']:.1f}s")
        print(f"         Tokens: {operational_metrics['prompt-tokens']} prompt + {operational_metrics['completion-tokens']} completion\n")

print("="*70)
print("\n✅ All queries executed successfully")
print(f"   Input saved to: {eval_input_path}\n")

# Evaluation 실행
print("="*70)
print("🔬 Running Evaluators\n")

print("   Evaluators:")
print("      • ToolCallAccuracyEvaluator")
print("      • IntentResolutionEvaluator")
print("      • TaskAdherenceEvaluator")
print("\n   ⚠️  참고: eastus 리전에서는 일부 RAI 평가자가 지원되지 않습니다.")
print("      (CodeVulnerability, ContentSafety, IndirectAttack는 제외)\n")
print("   ⏳ This may take 1-2 minutes...\n")

# OperationalMetricsEvaluator 정의
class OperationalMetricsEvaluator:
    """Propagate operational metrics to the final evaluation results"""
    def __init__(self):
        pass
    def __call__(self, *, metrics: dict, **kwargs):
        return metrics

# Evaluation 실행 (로컬)
print("   💡 평가 결과를 로컬에 저장합니다.\n")

results = evaluate(
    evaluation_name="foundry-agent-evaluation",
    data=eval_input_path,
    evaluators={
        "operational_metrics": OperationalMetricsEvaluator(),
        "tool_call_accuracy": ToolCallAccuracyEvaluator(model_config=model_config),
        "intent_resolution": IntentResolutionEvaluator(model_config=model_config),
        "task_adherence": TaskAdherenceEvaluator(model_config=model_config),
        # eastus 리전에서 지원되지 않는 평가자들은 제외
        # "code_vulnerability": CodeVulnerabilityEvaluator(credential=credential, azure_ai_project=project_endpoint),
        # "content_safety": ContentSafetyEvaluator(credential=credential, azure_ai_project=project_endpoint),
        # "indirect_attack": IndirectAttackEvaluator(credential=credential, azure_ai_project=project_endpoint)
    },
    output_path=eval_output_path,
    # azure_ai_project 파라미터 제거 (ML workspace 불필요)
)

print("✅ Evaluation completed!\n")
print(f"📁 결과 저장 위치: {eval_output_path}")
print(f"   다음 셀에서 결과를 확인하세요.\n")
print("="*70)

# Evaluation Agent 삭제
print("🧹 Cleaning up...")
ai_project.agents.delete_agent(eval_agent.id)
print(f"✅ Evaluation Agent deleted: {eval_agent.id}\n")

print("="*70)

## 6. Evaluation 결과 시각화

결과를 표와 차트로 시각화하여 분석합니다.

### 결과 확인 방법

**방법 1: 노트북에서 확인**
- 아래 셀을 실행하여 노트북에서 바로 결과를 확인합니다.

**방법 2: 터미널에서 확인**
- 터미널에서 다음 명령어를 실행하여 동일한 결과를 확인할 수 있습니다:
  ```bash
  python3 show_eval_results.py
  ```
- 이 스크립트는 프로젝트 루트에 자동으로 생성되어 있습니다.

In [ ]:
# Evaluation 결과 상세 시각화
import json
from pathlib import Path

SEPARATOR = "─" * 100
LINE = "=" * 100

def get_score_color(score, threshold=3.0):
    if score >= 4.5:
        return "\033[92m"
    elif score >= threshold:
        return "\033[93m"
    else:
        return "\033[91m"

def reset_color():
    return "\033[0m"

def get_score_indicator(score, threshold=3.0):
    if score >= 4.5:
        return "✅"
    elif score >= threshold:
        return "⚠️"
    else:
        return "❌"

def extract_query_text(query_input):
    if isinstance(query_input, list):
        for item in query_input:
            if isinstance(item, dict) and item.get("role") == "user":
                content = item.get("content", [])
                if isinstance(content, list) and len(content) > 0:
                    return content[0].get("text", "")
    return ""

def extract_response_text(response):
    if isinstance(response, list):
        for item in response:
            if isinstance(item, dict) and item.get("role") == "assistant":
                content = item.get("content", [])
                if isinstance(content, list) and len(content) > 0:
                    return content[0].get("text", "")
    return ""

eval_output_path = Path("evals/eval-output.json")

print(LINE)
print("📊 AGENT EVALUATION RESULTS - 상세 분석 리포트")
print(LINE, "\n")

if not eval_output_path.exists():
    print("❌ 평가 결과 파일이 없습니다.")
    print(f"   파일 경로: {eval_output_path.absolute()}")
    print("\n   먼저 06_evaluate_agents.ipynb의 셀 5를 실행하세요.\n")
else:
    with open(eval_output_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    metrics = data.get("metrics", {})
    rows = data.get("rows", [])
    
    # 섹션 1: 전체 평균 점수
    print("⭐ 전체 평균 성능 점수")
    print(LINE)
    scores_config = [
        ("Intent Resolution", "intent_resolution.intent_resolution", "의도 파악", 3.0),
        ("Task Adherence", "task_adherence.task_adherence", "작업 충실도", 3.0),
    ]
    
    for name, key, desc, threshold in scores_config:
        if key in metrics:
            score = metrics[key]
            color = get_score_color(score, threshold)
            reset = reset_color()
            indicator = get_score_indicator(score, threshold)
            stars = "★" * int(score) + "☆" * (5 - int(score))
            bar = "█" * int(score * 4) + "░" * (20 - int(score * 4))
            
            print(f"{indicator} {name:20} {color}{score:.2f}/5.0{reset}  {stars}")
            print(f"     {desc:20} [{bar}]")
            if score < threshold:
                print(f"     {color}⚠️ 임계값 미달 (기준: {threshold:.1f}){reset}")
            print()
    
    # 섹션 2: 운영 메트릭
    print("\n⚡ 운영 메트릭 (평균)")
    print(LINE)
    
    operational_keys = [
        ("operational_metrics.server-run-duration-in-seconds", "서버 실행 시간", "s"),
        ("operational_metrics.client-run-duration-in-seconds", "클라이언트 실행 시간", "s"),
        ("operational_metrics.prompt-tokens", "프롬프트 토큰", "tokens"),
        ("operational_metrics.completion-tokens", "완성 토큰", "tokens"),
    ]
    
    total_tokens = 0
    for key, desc, unit in operational_keys:
        if key in metrics:
            value = metrics[key]
            if unit == "tokens":
                print(f"  {desc:30} {int(value):>10,} {unit}")
                total_tokens += value
            else:
                print(f"  {desc:30} {value:>10.2f} {unit}")
    
    if total_tokens > 0:
        print(f"  {'총 토큰 사용량':30} {int(total_tokens):>10,} tokens")
        cost = (total_tokens / 1000) * 0.0025
        print(f"  {'예상 비용 (GPT-4o)':30} ${cost:>9.4f}")
    print()
    
    # 섹션 3: 개별 쿼리 상세 결과
    if rows:
        print("\n📋 쿼리별 상세 결과")
        print(LINE)
        
        for idx, row in enumerate(rows, 1):
            query = extract_query_text(row.get("inputs.query", []))
            response = extract_response_text(row.get("inputs.response", []))
            ground_truth = row.get("inputs.metrics.ground-truth", "")
            
            print(f"\n{SEPARATOR}")
            print(f"🔍 Query #{idx}")
            print(SEPARATOR)
            
            print("\n💬 사용자 질문:")
            print(f"   {query}")
            
            if ground_truth:
                print("\n📌 예상 동작 (Ground Truth):")
                print(f"   {ground_truth}")
            
            if response:
                print("\n🤖 Agent 응답 (요약):")
                response_preview = response[:200] if len(response) > 200 else response
                lines_shown = 0
                for line in response_preview.split("\n"):
                    if line.strip() and lines_shown < 3:
                        print(f"   {line.strip()}")
                        lines_shown += 1
                if len(response) > 200:
                    print(f"   ... (총 {len(response):,}자)")
            
            print("\n📊 평가 점수:")
            
            intent = row.get("outputs.intent_resolution.intent_resolution", "N/A")
            task = row.get("outputs.task_adherence.task_adherence", "N/A")
            tool = row.get("outputs.tool_call_accuracy.tool_call_accuracy", "N/A")
            
            intent_threshold = row.get("outputs.intent_resolution.intent_resolution_threshold", 3)
            task_threshold = row.get("outputs.task_adherence.task_adherence_threshold", 3)
            
            if isinstance(intent, (int, float)):
                color = get_score_color(intent, intent_threshold)
                reset = reset_color()
                indicator = get_score_indicator(intent, intent_threshold)
                print(f"   {indicator} Intent Resolution:  {color}{intent:.1f}/5.0{reset} (임계값: {intent_threshold})")
            else:
                print(f"   • Intent Resolution:  {intent}")
            
            if isinstance(task, (int, float)):
                color = get_score_color(task, task_threshold)
                reset = reset_color()
                indicator = get_score_indicator(task, task_threshold)
                print(f"   {indicator} Task Adherence:     {color}{task:.1f}/5.0{reset} (임계값: {task_threshold})")
            else:
                print(f"   • Task Adherence:     {task}")
            
            print(f"   • Tool Call Accuracy: {tool}")
            
            # 평가 이유
            print("\n💡 평가 상세:")
            
            intent_reason = row.get("outputs.intent_resolution.intent_resolution_reason", "")
            task_reason = row.get("outputs.task_adherence.task_adherence_reason", "")
            tool_reason = row.get("outputs.tool_call_accuracy.tool_call_accuracy_reason", "")
            
            if intent_reason:
                print("\n   [Intent Resolution 평가 이유]")
                for sentence in intent_reason.split(". "):
                    if sentence.strip():
                        print(f"   • {sentence.strip()}.")
            
            if task_reason:
                print("\n   [Task Adherence 평가 이유]")
                for sentence in task_reason.split(". "):
                    if sentence.strip():
                        print(f"   • {sentence.strip()}.")
            
            if tool_reason:
                print("\n   [Tool Call Accuracy 평가 이유]")
                for sentence in tool_reason.split(". "):
                    if sentence.strip():
                        print(f"   • {sentence.strip()}.")
            
            duration = row.get("outputs.operational_metrics.client-run-duration-in-seconds", 0)
            prompt_tokens = row.get("outputs.operational_metrics.prompt-tokens", 0)
            completion_tokens = row.get("outputs.operational_metrics.completion-tokens", 0)
            
            print("\n⏱️  성능 메트릭:")
            print(f"   • 실행 시간: {duration:.2f}초")
            print(f"   • 토큰 사용: {prompt_tokens:,} (입력) + {completion_tokens:,} (출력) = {prompt_tokens + completion_tokens:,} (총)")
            
            issues = []
            if isinstance(intent, (int, float)) and intent < intent_threshold:
                issues.append(f"Intent Resolution 점수 낮음 ({intent:.1f} < {intent_threshold})")
            if isinstance(task, (int, float)) and task < task_threshold:
                issues.append(f"Task Adherence 점수 낮음 ({task:.1f} < {task_threshold})")
            
            if issues:
                print(f"\n{get_score_color(1.0, 3.0)}⚠️  발견된 문제:{reset_color()}")
                for issue in issues:
                    print(f"   • {issue}")
        
        # 섹션 4: 통계 요약
        print(f"\n{SEPARATOR}\n")
        print("\n📈 통계 요약 및 분석")
        print(LINE)
        
        intent_scores = []
        task_scores = []
        durations = []
        total_tokens_list = []
        failed_queries = []
        
        for idx, row in enumerate(rows, 1):
            intent = row.get("outputs.intent_resolution.intent_resolution")
            task = row.get("outputs.task_adherence.task_adherence")
            duration = row.get("outputs.operational_metrics.client-run-duration-in-seconds", 0)
            prompt = row.get("outputs.operational_metrics.prompt-tokens", 0)
            completion = row.get("outputs.operational_metrics.completion-tokens", 0)
            
            intent_threshold = row.get("outputs.intent_resolution.intent_resolution_threshold", 3)
            task_threshold = row.get("outputs.task_adherence.task_adherence_threshold", 3)
            
            if isinstance(intent, (int, float)):
                intent_scores.append(intent)
                if intent < intent_threshold:
                    query = extract_query_text(row.get("inputs.query", []))
                    failed_queries.append((idx, "Intent Resolution", intent, query[:50]))
            
            if isinstance(task, (int, float)):
                task_scores.append(task)
                if task < task_threshold:
                    query = extract_query_text(row.get("inputs.query", []))
                    failed_queries.append((idx, "Task Adherence", task, query[:50]))
            
            if duration:
                durations.append(duration)
            total_tokens_list.append(prompt + completion)
        
        if intent_scores:
            avg_intent = sum(intent_scores) / len(intent_scores)
            color = get_score_color(avg_intent, 3.0)
            reset = reset_color()
            pass_count = len([s for s in intent_scores if s >= 3.0])
            
            print("\n📊 Intent Resolution (의도 파악)")
            print(f"   평균: {color}{avg_intent:.2f}/5.0{reset}")
            print(f"   최고: {max(intent_scores):.1f}  |  최저: {min(intent_scores):.1f}")
            print(f"   합격률: {pass_count}/{len(intent_scores)} ({pass_count/len(intent_scores)*100:.1f}%)")
        
        if task_scores:
            avg_task = sum(task_scores) / len(task_scores)
            color = get_score_color(avg_task, 3.0)
            reset = reset_color()
            pass_count = len([s for s in task_scores if s >= 3.0])
            
            print("\n📊 Task Adherence (작업 충실도)")
            print(f"   평균: {color}{avg_task:.2f}/5.0{reset}")
            print(f"   최고: {max(task_scores):.1f}  |  최저: {min(task_scores):.1f}")
            print(f"   합격률: {pass_count}/{len(task_scores)} ({pass_count/len(task_scores)*100:.1f}%)")
        
        if durations:
            print("\n⏱️  실행 시간")
            print(f"   평균: {sum(durations)/len(durations):.2f}초")
            print(f"   최대: {max(durations):.2f}초  |  최소: {min(durations):.2f}초")
        
        if total_tokens_list:
            avg_tokens = sum(total_tokens_list) / len(total_tokens_list)
            total_all_tokens = sum(total_tokens_list)
            
            print("\n💰 토큰 사용량")
            print(f"   평균: {avg_tokens:,.0f} tokens/query")
            print(f"   총합: {total_all_tokens:,} tokens")
            print(f"   최대: {max(total_tokens_list):,}  |  최소: {min(total_tokens_list):,}")
            print(f"   예상 비용 (GPT-4o): ${(total_all_tokens / 1000) * 0.0025:.4f}")
        
        if failed_queries:
            print(f"\n{get_score_color(1.0, 3.0)}⚠️  개선이 필요한 쿼리 ({len(failed_queries)}개){reset_color()}")
            print(SEPARATOR)
            
            seen = set()
            for idx, metric, score, query in failed_queries:
                key = (idx, metric)
                if key not in seen:
                    seen.add(key)
                    print(f"   Query #{idx}: {metric} = {score:.1f}")
                    print(f"   └─ {query}...")
                    print()
        else:
            print("\n✅ 모든 쿼리가 임계값을 통과했습니다!")
    
    print(f"\n{LINE}")
    print(f"✅ 총 {len(rows)}개 쿼리 평가 완료")
    print(f"📁 상세 JSON: {eval_output_path.absolute()}")
    print(f"💡 터미널에서도 실행 가능: python3 show_eval_results.py")
    print(f"{LINE}\n")

## 7. Evaluation 메트릭 해석 가이드

### 메트릭 해석

**Operational Metrics:**
- `server-run-duration-in-seconds`: 서버에서 Agent 실행 시간
- `client-run-duration-in-seconds`: 클라이언트에서 측정한 총 소요 시간
- `prompt-tokens`: 입력 토큰 수
- `completion-tokens`: 생성된 토큰 수

**Performance Metrics:**
- `tool_call_accuracy.*`: 도구 호출 정확도 (1-5점)
- `intent_resolution.*`: 의도 파악 정확도 (1-5점)
- `task_adherence.*`: 작업 준수도 (1-5점)

**점수 해석:**
- 5점: 완벽함
- 4점: 좋음
- 3점: 보통
- 2점: 개선 필요
- 1점: 매우 나쁨

---

## 8. 다음 단계 및 Agent 개선

### Agent 개선 방법

1. **낮은 점수 분석**
   - 어떤 쿼리에서 점수가 낮았는지 확인
   - 어떤 evaluator에서 문제가 발생했는지 파악

2. **프롬프트 개선**
   - Agent instructions를 더 명확하게 작성
   - 예시 추가
   - 제약사항 명시

3. **기능 개선**
   - Tool 추가 또는 개선
   - RAG 지식 베이스 보강
   - Multi-Agent 조정

4. **재평가**
   - 동일한 쿼리로 재평가
   - 점수 변화 비교
   - 지속적인 개선

### 참고 자료

- [Azure AI Foundry Agent Evaluation](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/agent-evaluate-sdk)
- [Built-in Evaluators](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/evaluate-sdk)
- [Evaluation Best Practices](https://learn.microsoft.com/azure/ai-foundry/concepts/evaluation-approach-gen-ai)

---

## 완료!

축하합니다! Agent Evaluation을 성공적으로 완료했습니다. 🎉